In [21]:
import mlc_func as mlcf
from mlc_func.elf.utils import hdf5_to_elfs
from mlc_func.elf.geom import make_complex, rotate_tensor
from mlc_func.elf.real_space import orient_elfs
from ase.io import read, write
from h5py import File
import json
import numpy as np

In [26]:
def change_alignment(path, traj_path, new_method, save_as = None):
    elfs = hdf5_to_elfs(path)
    atoms = read(traj_path, ':')
    
    with File(path) as file:
        basis = json.loads(file.attrs['basis'])
        
    if new_method == basis['alignment']:
        raise Exception('Already aligned with method: ' + new_method)
    
    #Rotate to neutral
    for i, elf_system in enumerate(elfs):
        for j, elf in enumerate(elf_system):
            elfs[i][j].value = rotate_tensor(make_complex(elf.value, basis['n_rad_' + elf.species.lower()],
                                                       basis['n_l_' + elf.species.lower()]), elf.angles, 
                                             inverse = False)   
            elfs[i][j].angles = np.array([0,0,0])
            elfs[i][j].unitcell = atoms[i].get_cell()
    
    oriented_elfs = []
    
    for elfs_system, atoms_system in zip(elfs, atoms):
        oriented_elfs.append(orient_elfs(elfs_system,atoms_system,new_method))
    return oriented_elfs

In [35]:
elfs = change_alignment('../data/PBE/szp/basis_nocore_elf.json_processed/64_processed.hdf5',
                 '../data/PBE/szp/64.traj',
                 'nn')

{'r_o_o': 1.0, 'r_i_o': 0.0, 'r_i_h': 0.0, 'r_o_h': 1.5, 'n_rad_o': 2, 'n_rad_h': 2, 'n_l_o': 3, 'n_l_h': 2, 'gamma_o': 0, 'gamma_h': 0, 'alignment': 'elf'}


In [45]:
elfs[10][0].value

array([ 2.34135976e+02,  5.84726550e+00,  3.94677413e-01,  4.32113833e+00,
       -7.62870950e-01, -1.12050613e-01, -1.69125290e+00, -3.51349358e-01,
        5.38822711e-01,  8.74099773e+01, -5.02862195e+00, -3.93007756e-02,
       -3.87229920e+00,  4.25907467e+00,  3.29404177e-01,  6.58705400e+00,
       -2.08814562e-01, -2.21381150e+00])

In [36]:
elfs_orig = hdf5_to_elfs('../data/PBE/szp/basis_nocore_nn.json_processed/64_processed.hdf5')

{'r_o_o': 1.0, 'r_i_o': 0.0, 'r_i_h': 0.0, 'r_o_h': 1.5, 'n_rad_o': 2, 'n_rad_h': 2, 'n_l_o': 3, 'n_l_h': 2, 'gamma_o': 0, 'gamma_h': 0, 'alignment': 'nn'}


In [46]:
elfs_orig[10][0].value

array([ 2.34135976e+02,  5.84726550e+00,  3.94677413e-01,  4.32113833e+00,
       -7.62870950e-01, -1.12050613e-01, -1.69125290e+00, -3.51349358e-01,
        5.38822711e-01,  8.74099773e+01, -5.02862195e+00, -3.93007756e-02,
       -3.87229920e+00,  4.25907467e+00,  3.29404177e-01,  6.58705400e+00,
       -2.08814562e-01, -2.21381150e+00])